In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
merge_df = pd.read_csv('./prepped_data/merged_bdinfo_floor_info_features.csv')
print(merge_df.shape)
merge_df.head()

(718969, 255)
Wall time: 4.57 s


,지번주소,기준년월,기준년월_최초사용승인일,주소별_최초생성일자,지붕_(철근)콘크리트_있음,지붕_기타지붕_있음,지붕_기와_있음,지붕_슬레이트_있음,최빈지붕,주_용도_부대시설_있음,주_용도_주차장_있음,주_용도_복리시설_있음,주_용도_사무소_있음,주_용도_소매점_있음,주_용도_차고_있음,주_용도_일반음식점_있음,주_용도_기타제1종근생활시설_있음,주_용도_기타제2종근생활시설_있음,주_용도_기타근린생활시설_있음,주_용도_기타창고시설_있음,주_용도_기타판매시설_있음,주_용도_기타일반업무시설_있음,주_용도_기타사무소_있음,주_용도_유치원_있음,주_용도_노인복지시설_있음,주_용도_치과의원_있음,주_용도_의원_있음,주_용도_제조업소_있음,주_용도_대피소_있음,주_용도_휴게음식점_있음,주_용도_학원_있음,주_용도_생활편익시설_있음,주_용도_세탁소_있음,주_용도_창고_있음,주_용도_독서실_있음,주_용도_기타노유자시설_있음,주_용도_부동산중개업소_있음,주_용도_기원_있음,주_용도_기타교육연구시설_있음,주_용도_이(미)용원_있음,주_용도_멀티미디어문화컨텐츠설비제공업소_있음,주_용도_기타자동차관련시설_있음,주_용도_어린이집_있음,주_용도_한의원_있음,주_용도_금융업소_있음,주_용도_일반게임제공업의시설_있음,주_용도_파출소_있음,주_용도_인터넷컴퓨터게임시설제공업소_있음,주_용도_상점_있음,주_용도_게임제공업소_있음,주_용도_기타교육연구및복지시설_있음,주_용도_출판사_있음,주_용도_일반목욕장_있음,주_용도_공중화장실_있음,주_용도_부동산중개사무소_있음,주_용도_미용원_있음,주_용도_기타아동관련시설_있음,주_용도_기타공장_있음,주_용도_수리점_있음,주_용도_영유아보육시설_있음,주_용도_연구소_있음,주_용도_체육도장_있음,주_용도_지역아동센터_있음,주_용도_체육장_있음,주_용도_기타종교집회장_있음,주_용도_기타 운동시설_있음,주_용도_노래연습장_있음,주_용도_당구장_있음,주_용도_기타의료시설_있음,주_용도_아동복지시설_있음,주_용도_의약품도매점_있음,주_용도_교회_있음,주_용도_단란주점_있음,주_용도_조산원_있음,주_용도_침술원_있음,주_용도_표구점_있음,주_용도_목욕장_있음,주_용도_기타운동시설_있음,주_용도_기타문화및집회시설_있음,주_용도_기타전시장_있음,주_용도_사진관_있음,주_용도_기타일반숙박시설_있음,주_용도_체력단련장_있음,주_용도_가족호텔_있음,주_용도_조산소_있음,주_용도_접골원_있음,주_용도_기타생활권수련시설_있음,주_용도_기타공연장_있음,주_용도_탁구장_있음,주_용도_시장_있음,주_용도_직업훈련소_있음,주_용도_기타집회장_있음,주_용도_골프연습장_있음,주_용도_교육(연수)원_있음,주_용도_결혼상담소_있음,주_용도_인터넷컴퓨터게임시설제공업의시설_있음,주_용도_기타종교시설_있음,주_용도_사회복지시설_있음,주_용도_기타공공시설_있음,주_용도_기타판매및영업시설_있음,주_용도_사찰_있음,주_용도_의약품판매소_있음,주_용도_의료기기판매소_있음,주_용도_청소년게임제공업의시설_있음,주_용도_이용원_있음,주_용도_서점(1종근.생미해당)_있음,주_용도_기타위락시설_있음,주_용도_마을공동시설_있음,주_용도_기타위험물저장처리시설_있음,주_용도_변전소_있음,주_용도_병원_있음,주_용도_동물병원_있음,주_용도_유흥주점_있음,주_용도_예식장_있음,주_용도_일반업무시설_있음,주_용도_기타발전시설_있음,주_용도_수퍼마켓_있음,주_용도_공공도서관_있음,주_용도_자동차영업소_있음,주_용도_제1종근린생활시설_있음,주_용도_분뇨처리시설_있음,주_용도_기타소매시장_있음,주_용도_온실_있음,주_용도_소매시장_있음,주_용도_안마시술소_있음,주_용도_야외극장_있음,주_용도_창고시설_있음,주_용도_에어로빅장_있음,주_용도_제2종근린생활시설_있음,주_용도_극장_있음,주_용도_한방병원_있음,주_용도_극장(영화관)_있음,주_용도_기타병원_있음,주_용도_제과점_있음,주_용도_교육연구시설_있음,주_용도_대형백화점_있음,주_용도_기타분뇨쓰레기처리시설_있음,주_용도_총포판매사_있음,주_용도_일반공장_있음,주_용도_기타공공업무시설_있음,주_용도_정수장_있음,주_용도_일반창고_있음,주_용도_운동장시설_있음,주_용도_여관_있음,주_용도_대형점_있음,주_용도_고시원_있음,주_용도_유원시설업의시설_있음,주_용도_무도장_있음,주_용도_치과병원_있음,주_용도_안마원_있음,주_용도_관광호텔_있음,주_용도_도서관_있음,주_용도_비디오물감상실_있음,주_용도_볼링장_있음,주_용도_교육원_있음,주_용도_마을공동작업소_있음,주_용도_산부인과병원_있음,주_용도_대학_있음,주_용도_주점영업_있음,주_용도_소개업_있음,주_용도_요양병원_있음,주_용도_수족관_있음,주_용도_매매장_있음,주_용도_테니스장_있음,주_용도_동사무소_있음,주_용도_무도장(학원)_있음,주_용도_대학교_있음,주_용도_호텔_있음,주_용도_운동시설_있음,주_용도_복합유통게임제공업의시설_있음,주_용도_영화관_있음,주_용도_기숙사_있음,주_용도_체육관_있음,주_용도_도매시장_있음,주_용도_정비공장_있음,주_용도_복합유통게임제공업소_있음,주_용도_회의장_있음,주_용도_총포판매소_있음,주_용도_미술관_있음,주_용도_백화점_있음,주_용도_기타학교_있음,주_용도_공관_있음,주_용도_군사시설_있음,주_용도_방송국_있음,주_용도_기타제2종근린생활시설_있음,주_용도_기타제1종근린생활시설_있음,주_용도_기타동식물관련시설_있음,주_용도_고압가스충전저장소_있음,주_용도_검사장_있음,주_용도_초등학교_있음,주_용도_기타관광숙박시설_있음,주_용도_세차장_있음,주_용도_성당_있음,주_용도_축사_있음,주_용도_기타관광휴게시설_있음,주_용도_위험물취급소_있음,주_용도_석유판매소_있음,주_용도_물놀이형시설_있음,주_용도_보건소_있음,주_용도_고등학교_있음,주_용도_유해공장_있음,주_용도_주유소_있음,주_용도_다중생활시설_있음,주_용도_일반숙박시설_있음,주_용도_호스텔_있음,주_용도_박물관_있음,주_용도_생활숙박시설_있음,주_용도_청소년문화의집_있음,주_용도_근린생활시설_있음,주_용도_산후조리원_있음,주_용도_교습소_있음,주_용도_청소년게임제공업소_있음,주_용도_액화가스판매소_있음,주_용도_야외음악당_있음,주_용도_신문사_있음,주_용도_음악당_있음,주_용도_수도원_있음,주_용도_냉동창고_있음,주_용도_놀이형시설_있음,주_용도_기타게임시설_있음,주_용도_제실_있음,주_용도_자치단체청사_있음,주_용도_비디오물소극장_있음,주_용도_국방ㆍ군사시설_있음,주_용도_지구대_있음,주_용도_기도원_있음,주_용도_롤러스케이트장_있음,주_용도_소형호텔_있음,주_용도_수영장_있음,주_용도_동물미용실_있음,주_용도_도료류판매소_있음,주_용도_물품 제조공장_있음,주_용도_물품 인쇄공장_있음,최빈용도,구조_철근콘크리트구조_있음,구조_벽돌구조_있음,구조_철골콘크리트구조_있음,구조_기타조적구조_있음,구조_경량철골구조_있음,구조_철골철근콘크리트구조_있음,구조_일반철골구조_있음,구조_기타강구조_있음,구조_프리케스트콘크리트구조_있음,구조_블록구조_있음,구조_기타구조_있음,구조_기타콘크리트구조_있음,구조_강파이프구조_있음,구조_일반목구조_있음,구조_조적구조_있음,구조_기타철골철근콘크리트구조_있음,구조_석구조_있음,구조_철골철근콘크리트합성구조_있음,구조_라멘조_있음,구조_조립식판넬조_있음,최빈구조
0,서울특별시 강남구 개포동 1163-4,201412,20130307.0,20130313,1,0,0,0,(철근)콘크리트,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [3]:
def date_to_year(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        return int(str(x)[:4])

In [4]:
merge_df['기준년월'] = merge_df['기준년월'].apply(date_to_year)
merge_df['기준년월_최초사용승인일'] = merge_df['기준년월_최초사용승인일'].apply(date_to_year)
merge_df['주소별_최초생성일자'] = merge_df['주소별_최초생성일자'].apply(date_to_year)

In [5]:
merge_df.rename(columns={'기준년월':'기준년도', '기준년월_최초사용승인일':'사용승인년도', '주소별_최초생성일자':'생성년도'}, inplace=True)

In [6]:
merge_df[['기준년도', '사용승인년도', '생성년도']].isna().sum()

기준년도         0
사용승인년도    1130
생성년도         0
dtype: int64

In [7]:
merge_df['사용승인년도'].fillna(merge_df['생성년도'], inplace=True)
merge_df['사용승인년도'].isna().sum()

0

In [8]:
merge_df['기준년도'] = merge_df['기준년도'].astype('int')
merge_df['사용승인년도'] = merge_df['사용승인년도'].astype('int')
merge_df['생성년도'] = merge_df['생성년도'].astype('int')

In [9]:
merge_df.drop(columns=['생성년도'], inplace=True)

In [10]:
def create_full_df_iteratively(df):
    # df is a pandas dataframe
    
    import pandas as pd
    import numpy as np
    import gc
    
    df = df.copy()
    
    dfs_list = []
    count = 0
    for addr in df['지번주소'].unique():
        addr_df = df[df['지번주소'] == addr]
        
        min_record_year = addr_df['기준년도'].min()
        max_record_year = addr_df['기준년도'].max()
        unique_record_years = addr_df['기준년도'].unique()

        unique_approv_years = sorted(addr_df['사용승인년도'].unique().tolist())
        min_approved_year = addr_df['사용승인년도'].min()
        max_approved_year = addr_df['사용승인년도'].max()

        if min_approved_year <= 2006:
            min_year = 2006
        else:
            min_year = min_approved_year

        years_for_concat = [y for y in range(min_year, 2021) if y not in unique_record_years]
        if years_for_concat == []:
            dfs_list.append(addr_df)
            continue

        rows_list = []
        for y in years_for_concat:
            empty_row = addr_df.iloc[[0],:]
            empty_row['기준년도'] = y
            empty_row.iloc[0, 2:] = np.nan
            rows_list.append(empty_row)

        empty_df = pd.concat(rows_list)

        addr_concat_df = pd.concat([addr_df, empty_df]).sort_values(['기준년도']).reset_index(drop=True)

        addr_concat_df.fillna(method='ffill', inplace=True)
        addr_concat_df.fillna(method='bfill', inplace=True)
        
        dfs_list.append(addr_concat_df)
        count += 1
        if count % 5000 == 0:
            gc.collect()
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
    
    return concat_df

In [11]:
def parallelize(df, func, num_processors=6):
    # df is a pandas dataframe
    # func is the function to use
    # num_processors is the number of cpu cores
    
    data_split = []
    unique_addrs = df['지번주소'].unique()
    divided = int(len(unique_addrs) / num_processors)
    for i in range(num_processors):
        if i < (num_processors - 1):
            picked_addrs = unique_addrs[divided*i:divided*(i+1)]
        else:
            picked_addrs = unique_addrs[divided*i:]
        data_split.append(df[df['지번주소'].isin(picked_addrs)])
            
    pool = mp.Pool(num_processors)
    concat_df = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return concat_df

In [12]:
%%time
completed_df = parallelize(merge_df, create_full_df_iteratively)
print(completed_df.shape)
completed_df.head()

(1469986, 254)
Wall time: 2h 34min


,지번주소,기준년도,사용승인년도,지붕_(철근)콘크리트_있음,지붕_기타지붕_있음,지붕_기와_있음,지붕_슬레이트_있음,최빈지붕,주_용도_부대시설_있음,주_용도_주차장_있음,주_용도_복리시설_있음,주_용도_사무소_있음,주_용도_소매점_있음,주_용도_차고_있음,주_용도_일반음식점_있음,주_용도_기타제1종근생활시설_있음,주_용도_기타제2종근생활시설_있음,주_용도_기타근린생활시설_있음,주_용도_기타창고시설_있음,주_용도_기타판매시설_있음,주_용도_기타일반업무시설_있음,주_용도_기타사무소_있음,주_용도_유치원_있음,주_용도_노인복지시설_있음,주_용도_치과의원_있음,주_용도_의원_있음,주_용도_제조업소_있음,주_용도_대피소_있음,주_용도_휴게음식점_있음,주_용도_학원_있음,주_용도_생활편익시설_있음,주_용도_세탁소_있음,주_용도_창고_있음,주_용도_독서실_있음,주_용도_기타노유자시설_있음,주_용도_부동산중개업소_있음,주_용도_기원_있음,주_용도_기타교육연구시설_있음,주_용도_이(미)용원_있음,주_용도_멀티미디어문화컨텐츠설비제공업소_있음,주_용도_기타자동차관련시설_있음,주_용도_어린이집_있음,주_용도_한의원_있음,주_용도_금융업소_있음,주_용도_일반게임제공업의시설_있음,주_용도_파출소_있음,주_용도_인터넷컴퓨터게임시설제공업소_있음,주_용도_상점_있음,주_용도_게임제공업소_있음,주_용도_기타교육연구및복지시설_있음,주_용도_출판사_있음,주_용도_일반목욕장_있음,주_용도_공중화장실_있음,주_용도_부동산중개사무소_있음,주_용도_미용원_있음,주_용도_기타아동관련시설_있음,주_용도_기타공장_있음,주_용도_수리점_있음,주_용도_영유아보육시설_있음,주_용도_연구소_있음,주_용도_체육도장_있음,주_용도_지역아동센터_있음,주_용도_체육장_있음,주_용도_기타종교집회장_있음,주_용도_기타 운동시설_있음,주_용도_노래연습장_있음,주_용도_당구장_있음,주_용도_기타의료시설_있음,주_용도_아동복지시설_있음,주_용도_의약품도매점_있음,주_용도_교회_있음,주_용도_단란주점_있음,주_용도_조산원_있음,주_용도_침술원_있음,주_용도_표구점_있음,주_용도_목욕장_있음,주_용도_기타운동시설_있음,주_용도_기타문화및집회시설_있음,주_용도_기타전시장_있음,주_용도_사진관_있음,주_용도_기타일반숙박시설_있음,주_용도_체력단련장_있음,주_용도_가족호텔_있음,주_용도_조산소_있음,주_용도_접골원_있음,주_용도_기타생활권수련시설_있음,주_용도_기타공연장_있음,주_용도_탁구장_있음,주_용도_시장_있음,주_용도_직업훈련소_있음,주_용도_기타집회장_있음,주_용도_골프연습장_있음,주_용도_교육(연수)원_있음,주_용도_결혼상담소_있음,주_용도_인터넷컴퓨터게임시설제공업의시설_있음,주_용도_기타종교시설_있음,주_용도_사회복지시설_있음,주_용도_기타공공시설_있음,주_용도_기타판매및영업시설_있음,주_용도_사찰_있음,주_용도_의약품판매소_있음,주_용도_의료기기판매소_있음,주_용도_청소년게임제공업의시설_있음,주_용도_이용원_있음,주_용도_서점(1종근.생미해당)_있음,주_용도_기타위락시설_있음,주_용도_마을공동시설_있음,주_용도_기타위험물저장처리시설_있음,주_용도_변전소_있음,주_용도_병원_있음,주_용도_동물병원_있음,주_용도_유흥주점_있음,주_용도_예식장_있음,주_용도_일반업무시설_있음,주_용도_기타발전시설_있음,주_용도_수퍼마켓_있음,주_용도_공공도서관_있음,주_용도_자동차영업소_있음,주_용도_제1종근린생활시설_있음,주_용도_분뇨처리시설_있음,주_용도_기타소매시장_있음,주_용도_온실_있음,주_용도_소매시장_있음,주_용도_안마시술소_있음,주_용도_야외극장_있음,주_용도_창고시설_있음,주_용도_에어로빅장_있음,주_용도_제2종근린생활시설_있음,주_용도_극장_있음,주_용도_한방병원_있음,주_용도_극장(영화관)_있음,주_용도_기타병원_있음,주_용도_제과점_있음,주_용도_교육연구시설_있음,주_용도_대형백화점_있음,주_용도_기타분뇨쓰레기처리시설_있음,주_용도_총포판매사_있음,주_용도_일반공장_있음,주_용도_기타공공업무시설_있음,주_용도_정수장_있음,주_용도_일반창고_있음,주_용도_운동장시설_있음,주_용도_여관_있음,주_용도_대형점_있음,주_용도_고시원_있음,주_용도_유원시설업의시설_있음,주_용도_무도장_있음,주_용도_치과병원_있음,주_용도_안마원_있음,주_용도_관광호텔_있음,주_용도_도서관_있음,주_용도_비디오물감상실_있음,주_용도_볼링장_있음,주_용도_교육원_있음,주_용도_마을공동작업소_있음,주_용도_산부인과병원_있음,주_용도_대학_있음,주_용도_주점영업_있음,주_용도_소개업_있음,주_용도_요양병원_있음,주_용도_수족관_있음,주_용도_매매장_있음,주_용도_테니스장_있음,주_용도_동사무소_있음,주_용도_무도장(학원)_있음,주_용도_대학교_있음,주_용도_호텔_있음,주_용도_운동시설_있음,주_용도_복합유통게임제공업의시설_있음,주_용도_영화관_있음,주_용도_기숙사_있음,주_용도_체육관_있음,주_용도_도매시장_있음,주_용도_정비공장_있음,주_용도_복합유통게임제공업소_있음,주_용도_회의장_있음,주_용도_총포판매소_있음,주_용도_미술관_있음,주_용도_백화점_있음,주_용도_기타학교_있음,주_용도_공관_있음,주_용도_군사시설_있음,주_용도_방송국_있음,주_용도_기타제2종근린생활시설_있음,주_용도_기타제1종근린생활시설_있음,주_용도_기타동식물관련시설_있음,주_용도_고압가스충전저장소_있음,주_용도_검사장_있음,주_용도_초등학교_있음,주_용도_기타관광숙박시설_있음,주_용도_세차장_있음,주_용도_성당_있음,주_용도_축사_있음,주_용도_기타관광휴게시설_있음,주_용도_위험물취급소_있음,주_용도_석유판매소_있음,주_용도_물놀이형시설_있음,주_용도_보건소_있음,주_용도_고등학교_있음,주_용도_유해공장_있음,주_용도_주유소_있음,주_용도_다중생활시설_있음,주_용도_일반숙박시설_있음,주_용도_호스텔_있음,주_용도_박물관_있음,주_용도_생활숙박시설_있음,주_용도_청소년문화의집_있음,주_용도_근린생활시설_있음,주_용도_산후조리원_있음,주_용도_교습소_있음,주_용도_청소년게임제공업소_있음,주_용도_액화가스판매소_있음,주_용도_야외음악당_있음,주_용도_신문사_있음,주_용도_음악당_있음,주_용도_수도원_있음,주_용도_냉동창고_있음,주_용도_놀이형시설_있음,주_용도_기타게임시설_있음,주_용도_제실_있음,주_용도_자치단체청사_있음,주_용도_비디오물소극장_있음,주_용도_국방ㆍ군사시설_있음,주_용도_지구대_있음,주_용도_기도원_있음,주_용도_롤러스케이트장_있음,주_용도_소형호텔_있음,주_용도_수영장_있음,주_용도_동물미용실_있음,주_용도_도료류판매소_있음,주_용도_물품 제조공장_있음,주_용도_물품 인쇄공장_있음,최빈용도,구조_철근콘크리트구조_있음,구조_벽돌구조_있음,구조_철골콘크리트구조_있음,구조_기타조적구조_있음,구조_경량철골구조_있음,구조_철골철근콘크리트구조_있음,구조_일반철골구조_있음,구조_기타강구조_있음,구조_프리케스트콘크리트구조_있음,구조_블록구조_있음,구조_기타구조_있음,구조_기타콘크리트구조_있음,구조_강파이프구조_있음,구조_일반목구조_있음,구조_조적구조_있음,구조_기타철골철근콘크리트구조_있음,구조_석구조_있음,구조_철골철근콘크리트합성구조_있음,구조_라멘조_있음,구조_조립식판넬조_있음,최빈구조
0,서울특별시 강남구 개포동 1163-4,2013,2013.0,1.0,0.0,0.0,0.0,(철근)콘크리트,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [13]:
completed_df.to_csv('./prepped_data/bdinfo_floor_info_features_full.csv', index=False)